Import Packages

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sb

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

California Schools
http://www.cde.ca.gov/ds/si/ds/fspubschls.asp

In [2]:
# All school data
school=pd.read_excel('schools.xlsx',header=0,skiprows=0)

In [3]:
school=school[['CDSCode','StatusType','County','District','School','Zip','Latitude','Longitude','ClosedDate','FundingType','Virtual']]

In [4]:
school.ClosedDate = pd.to_datetime(school.ClosedDate)


In [5]:
#school['ClosedYear'] = school.ClosedDate.year
school.set_index('ClosedDate', inplace = True)
school['ClosedYear'] = school.index.year


In [6]:
school.head(2)

,CDSCode,StatusType,County,District,School,Zip,Latitude,Longitude,FundingType,Virtual,ClosedYear
ClosedDate,,,,,,,,,,,
NaT,1100170000000,Active,Alameda,Alameda County Office of Education,NaN,94544-1136,37.658212,-122.09713,NaN,NaN,NaN
2015-07-31,1100170109835,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,94560-5359,37.521436,-121.99391,Directly funded,P,2015.0


In [7]:
#school=school[[(school.ClosedYear>2015)|(school.StatusType=='Active')]]
school2=school[school.ClosedYear>2015]
school1=school[school.StatusType=='Active']

In [8]:
school=pd.concat([school1,school2])


In [12]:
school['five_zip']=school['Zip'].str[:5]
school.head(2)

,CDSCode,StatusType,County,District,School,Zip,Latitude,Longitude,FundingType,Virtual,ClosedYear,five_zip
ClosedDate,,,,,,,,,,,,
NaT,1100170000000,Active,Alameda,Alameda County Office of Education,NaN,94544-1136,37.658212,-122.09713,NaN,NaN,NaN,94544
NaT,1100170112607,Active,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,94612-3355,37.804520,-122.26815,Directly funded,N,NaN,94612


In [10]:
#enrollment data
#Data on Teachers 
enr15=pd.read_csv('enr15.txt', sep='\t', lineterminator='\n')
enr15=enr15[['CDS_CODE','ETHNIC','GENDER','ENR_TOTAL']]
#enr15.RecID=enr15.CDS_CODE.astype(int)
enr15.head()

,CDS_CODE,ETHNIC,GENDER,ENR_TOTAL
0,1611190106401,5,M,6
1,1611190106401,5,F,12
2,1611190106401,3,F,1
3,1611190106401,7,M,12
4,1611190106401,3,M,2


	
Racial/ethnic designation. This field is coded as follows:

Code 0 = Not reported

Code 1 = American Indian or Alaska Native, Not Hispanic

Code 2 = Asian, Not Hispanic

Code 3 = Pacific Islander, Not Hispanic

Code 4 = Filipino, Not Hispanic

Code 5 = Hispanic or Latino

Code 6 = African American, not Hispanic

Code 7 = White, not Hispanic

Code 9 = Two or More Races, Not Hispanic

In [11]:
enr15.CDS_CODE.unique()

array([ 1611190106401,  1611190111765,  1611436105316, ..., 44698490000001,
       39754990000001, 44698490126920])

In [79]:
#enrollment sql
q = """
SELECT CDS_CODE AS cds,
SUM(CASE WHEN GENDER='M' THEN ENR_TOTAL ELSE 0 END)*1.0/SUM(ENR_TOTAL) AS perc_male,
SUM(CASE WHEN ETHNIC=7 THEN ENR_TOTAL ELSE 0 END)*1.0/SUM(ENR_TOTAL) AS perc_wh,
SUM(CASE WHEN ETHNIC=2 THEN ENR_TOTAL ELSE 0 END)*1.0/SUM(ENR_TOTAL) AS perc_as,
SUM(CASE WHEN ETHNIC=5 THEN ENR_TOTAL ELSE 0 END)*1.0/SUM(ENR_TOTAL) AS perc_his,
SUM(CASE WHEN ETHNIC=6 THEN ENR_TOTAL ELSE 0 END)*1.0/SUM(ENR_TOTAL) AS perc_aa,
SUM(ENR_TOTAL) AS total_enrollment
FROM enr15
GROUP BY cds
"""


In [80]:
school_enr = pysqldf(q)
school_enr['year']='2015'
school_enr.head()

,cds,perc_male,perc_wh,perc_as,perc_his,perc_aa,total_enrollment,year
0,1100170109835,0.440662,0.479301,0.237351,0.152714,0.079117,1087,2015
1,1100170112607,0.427848,0.027848,0.020253,0.468354,0.412658,395,2015
2,1100170118489,0.491803,0.008197,0.024590,0.647541,0.286885,244,2015
3,1100170123968,0.471204,0.125654,0.078534,0.450262,0.251309,191,2015
4,1100170124172,0.509728,0.089494,0.439689,0.046693,0.038911,257,2015


In [49]:
#income by zip code (2010 census)
income=pd.read_excel('income.xlsx',header=0,skiprows=0)
income.head()

,Zip,Median,Mean,Pop
0,1001,56662.5735,66687.8,16445
1,1002,49853.4177,75062.6,28069
2,1003,28462.0000,35121,8491
3,1005,75423.0000,82442,4798
4,1007,79076.3540,85802,12962



Staff credentials http://www.cde.ca.gov/ds/sd/df/filescertstaff.asp

Staff demographics http://www.cde.ca.gov/ds/sd/df/filesstaffdemo.asp

In [50]:
#Data on Teachers 
staff15=pd.read_csv('staff15.txt', sep='\t', lineterminator='\r')
staff15=staff15[['RecID','YearsTeaching','DistrictCode','GenderCode','EducationLevel']]
staff15.head(2)

,RecID,YearsTeaching,DistrictCode,GenderCode,EducationLevel
0,1206146,12,1964436,M,M
1,1222764,7,5472256,M,M


In [51]:
#Teachers by School
staffschool15=pd.read_csv('StaffSchool15.txt', sep='\t', lineterminator='\r',low_memory=False)

staffschool15=staffschool15[['RecID','SchoolCode','DistrictCode']]
staffschool15=staffschool15.dropna(subset=['SchoolCode','DistrictCode'])
staffschool15.SchoolCode=staffschool15.SchoolCode.astype(int)
staffschool15.RecID=staffschool15.RecID.astype(int)
staffschool15.DistrictCode=staffschool15.DistrictCode.astype(int)
staffschool15.head(2)

,RecID,SchoolCode,DistrictCode
0,1254553,6116255,4870581
1,1099255,6116255,4870581


In [52]:
q1  = """
SELECT
  sc.DistrictCode||sc.SchoolCode AS CDS,
  sc.SchoolCode,
  SUM(st.YearsTeaching) as total_years,
  COUNT(DISTINCT sc.RecID) AS num_staff,
  COUNT(DISTINCT CASE WHEN EducationLevel='M' THEN sc.RecID ELSE NULL END) AS num_staff_masters,
  SUM(st.YearsTeaching)*1.0/COUNT(DISTINCT sc.RecID) AS avg_years_teaching
    
FROM
  staffschool15 AS sc
LEFT JOIN
  staff15 st
    ON sc.RecId = st.RecId
    AND sc.DistrictCode=st.DistrictCode
GROUP BY 1,2
;
"""

In [53]:
school_teacher = pysqldf(q1)
school_teacher['year']='2015'
school_teacher.head()

,CDS,SchoolCode,total_years,num_staff,num_staff_masters,avg_years_teaching,year
0,10101080,0,2082,111,63,18.756757,2015
1,10101081030337,1030337,455,32,12,14.218750,2015
2,10101081030899,1030899,109,7,1,15.571429,2015
3,10101081030923,1030923,141,10,5,14.100000,2015
4,1010108109991,109991,236,17,2,13.882353,2015


high school grads data http://www.cde.ca.gov/ds/sd/sd/filesgrads.asp

In [54]:
grads15=pd.read_csv('grads2015.txt', sep='\t', lineterminator='\r')
grads15.CDS_CODE=grads15.CDS_CODE.astype(int)
grads15.head(2)

,CDS_CODE,ETHNIC,GENDER,GRADS,UC_GRADS,YEAR
0,1100170000000,2,M,12,1,1415
1,1100170000000,5,M,1,0,1415


In [55]:
q2  = """
SELECT
  CDS_CODE AS cds,
  SUM(GRADS) AS total_grads,
  SUM(UC_GRADS) AS uc_grads,
  SUM(CASE WHEN GENDER='M' THEN GRADS ELSE 0 END) AS male_grads,
  SUM(CASE WHEN GENDER='M' THEN UC_GRADS ELSE 0 END) AS male_uc_grads,
  SUM(CASE WHEN ETHNIC!=7 THEN GRADS ELSE 0 END) AS minority_grads,
  SUM(CASE WHEN ETHNIC!=7 THEN UC_GRADS ELSE 0 END) AS minority_uc_grads

FROM
  grads15
GROUP BY CDS_CODE
;
"""

In [56]:
school_grads = pysqldf(q2)
school_grads['year']='2015'
school_grads.head()

,cds,total_grads,uc_grads,male_grads,male_uc_grads,minority_grads,minority_uc_grads,year
0,1100170000000,333,129,153,54,288,125,2015
1,1316170000000,37,0,23,0,25,0,2015
2,1611190000000,834,427,405,196,643,321,2015
3,1611270000000,269,158,150,78,183,107,2015
4,1611430000000,821,515,371,207,508,253,2015


free / reduced meals  http://www.cde.ca.gov/ds/sd/sd/filessp.asp

In [57]:
frpm15=pd.read_excel('frpm1415.xls',sheetname='FRPM School-Level Data ',header=1,skiprows=0,)
frpm15.columns = [c.replace(' ', '_') for c in frpm15.columns]
frpm15.columns = [c.replace('%', '') for c in frpm15.columns]
frpm15.columns = [c.replace('-', '_') for c in frpm15.columns]
frpm15.columns = [c.replace('\n', '_') for c in frpm15.columns]
frpm15.columns = [c.replace('Percent_()__Eligible_Free__(K_12)', 'perc_eligible_free') for c in frpm15.columns]
frpm15.columns = [c.replace('Enrollment__(K_12)', 'enrollment_k12') for c in frpm15.columns]
frpm15.columns = [c.replace('Percent_()__Eligible_FRPM__(K_12)', 'perc_eligible_FRPM') for c in frpm15.columns]


frpm15.head(2)

,Academic_Year,County_Code,District_Code,School_Code,County_Name,District_Name,School_Name,District_Type,School_Type,Educational__Option_Type,...,Free_Meal__Count__(K_12),perc_eligible_free,FRPM_Count__(K_12),perc_eligible_FRPM,Enrollment__(Ages_5_17),Free_Meal__Count__(Ages_5_17),Percent_()__Eligible_Free__(Ages_5_17),FRPM_Count__(Ages_5_17),Percent_()__Eligible_FRPM__(Ages_5_17),2013_14__CALPADS_Fall_1__Certification_Status
0,2014-2015,1.0,10017.0,109835.0,Alameda,Alameda County Office of Education,FAME Public Charter,County Office of Education (COE),K-12 Schools (Public),Traditional,...,565.0,0.519779,715.0,0.657774,1070.0,553.0,0.516822,702.0,0.656075,Y
1,2014-2015,1.0,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,County Office of Education (COE),High Schools (Public),Traditional,...,186.0,0.470886,186.0,0.470886,376.0,182.0,0.484043,182.0,0.484043,Y


In [58]:
frpm15=frpm15[['County_Code','District_Code','School_Code','perc_eligible_free','perc_eligible_FRPM']]
frpm15=frpm15.dropna(subset=['School_Code','District_Code'])
frpm15.County_Code=frpm15.County_Code.astype(int)
frpm15.District_Code=frpm15.District_Code.astype(int)
frpm15.School_Code=frpm15.School_Code.astype(int)
frpm15['County_Code'] = frpm15.County_Code.map("{:02}".format)
frpm15['School_Code'] = frpm15.School_Code.map("{:07}".format)

frpm15['year']='2015'
frpm15.head()

,County_Code,District_Code,School_Code,perc_eligible_free,perc_eligible_FRPM,year
0,01,10017,0109835,0.519779,0.657774,2015
1,01,10017,0112607,0.470886,0.470886,2015
2,01,10017,0118489,0.549180,0.717213,2015
3,01,10017,0123968,0.591623,0.727749,2015
4,01,10017,0124172,0.054475,0.081712,2015


In [59]:
frpm15.County_Code.unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '57', '58'], dtype=object)

In [60]:
#df['period'] = df[['Year', 'quarter']].apply(lambda x: ''.join(x), axis=1)
frpm15.District_Code=frpm15.District_Code.astype(str)
frpm15.School_Code=frpm15.School_Code.astype(str)
frpm15.County_Code=frpm15.County_Code.astype(str)
frpm15['cds']=frpm15[['County_Code','District_Code','School_Code']].apply(lambda x: ''.join(x), axis=1)
#frpm15.cds=frpm15.cds.astype(int)
frpm15.head()

,County_Code,District_Code,School_Code,perc_eligible_free,perc_eligible_FRPM,year,cds
0,01,10017,0109835,0.519779,0.657774,2015,01100170109835
1,01,10017,0112607,0.470886,0.470886,2015,01100170112607
2,01,10017,0118489,0.549180,0.717213,2015,01100170118489
3,01,10017,0123968,0.591623,0.727749,2015,01100170123968
4,01,10017,0124172,0.054475,0.081712,2015,01100170124172


APR/ adequate yearly progress http://www.cde.ca.gov/ta/ac/ay/reclayout14.asp

In [61]:
    apr15=pd.read_excel('apr15.xlsx',header=0,skiprows=0)
    apr15.head(2)

,cds,rtype,type,sped,charter,sname,dname,cname,met_all,crit1,...,e_pp_app,m_pp_app,grad_app,ela_met,math_met,e_targ15,m_targ15,AvgDailyAttend,SchAttend,MetAttendTarg
0,0,X,NaN,NaN,NaN,State Total,State Total,NaN,Yes,26,...,NaN,NaN,Yes,Yes,Yes,100.0,100.0,96,NaN,Yes
1,1100170000000,D,1,NaN,NaN,NaN,Alameda County Office of Education,Alameda,No,1,...,NaN,NaN,Yes,No,No,100.0,100.0,NaN,NaN,NaN


In [62]:
apr15=apr15[['sname','cds','type','charter', 'met_all','crit1','crit2',
                   'e_enr','e_tst','e_prof','m_enr', 'm_tst','m_prof',
                    'ee_aa','et_aa','enp_aa','me_aa','mt_aa','mnp_aa',
                    'ee_wh','et_wh','enp_wh','me_wh','mt_wh','mnp_wh',
                    'ee_as','et_as','enp_as','me_as','mt_as','mnp_as',
                    'ee_hi','et_hi','enp_hi','me_hi','mt_hi','mnp_hi',
                    'e_pprof', #english % proficient
                    'm_pprof', #math % prof
                    'eppm_aa', #black english % prof
                    'mppm_aa', #black math % prof
                    'eppm_wh','mppm_wh', #white
                    'eppm_hi','eppm_hi', #hispanic
                    'eppm_as','mppm_as', #asian
                    'ela_met','math_met', #enlish/math met schoolwide (yes/no)
                    'grad15', #2015 graduation rates
                    'SchAttend', #avg attendance
                    'MetAttendTarg' #met attendance target (yes/no)
                   ]]
apr15.crit1=apr15.crit1.astype(int)
apr15.crit2=apr15.crit2.astype(int)
apr15['perc_crit']=apr15['crit1']*1.0/apr15['crit2']
apr15['year']='2015'
apr15.head()

,sname,cds,type,charter,met_all,crit1,crit2,e_enr,e_tst,e_prof,...,eppm_hi,eppm_as,mppm_as,ela_met,math_met,grad15,SchAttend,MetAttendTarg,perc_crit,year
0,State Total,0,NaN,NaN,Yes,26,26,3298149.0,3205026.0,1354011,...,--,--,--,Yes,Yes,80.95,NaN,Yes,1.000000,2015
1,NaN,1100170000000,1,NaN,No,1,9,190.0,71.0,0,...,--,--,--,No,No,82.88,NaN,NaN,0.111111,2015
2,FAME Public Charter,1100170109835,E,D,No,5,14,571.0,494.0,139,...,--,--,--,No,No,81.16,NaN,Yes,0.357143,2015
3,Envision Academy for Arts & Technology,1100170112607,H,D,Yes,3,3,90.0,82.0,31,...,--,--,--,Yes,Yes,66.07,NaN,NaN,1.000000,2015
4,Aspire California College Preparatory Academy,1100170118489,H,D,Yes,2,2,59.0,56.0,27,...,--,--,--,Yes,Yes,--,NaN,NaN,1.000000,2015


Truency http://www.cde.ca.gov/ds/sd/sd/fstd.asp

In [63]:
tru15=pd.read_csv('TruancyData.txt', sep='\t', lineterminator='\r')
tru15.columns = [c.replace(' ', '_') for c in tru15.columns]
tru15=tru15.dropna(subset=['Cds_Code'])
tru15.Cds_Code=tru15.Cds_Code.astype(int)
tru15.Cds_Code=tru15.Cds_Code.astype(str)
tru15.head(2)

,Year,Cds_Code,AggLevel,Name,Truant_Students,Census_Enrollment,Cumulative_Enrollment,Truancy_Rate
0,\n1415,1100170000000,D,Alameda County Office of Education,1694.0,3931.0,5411.0,31.3065
1,\n1415,1316090000000,D,California School for the Blind (State Special...,0.0,60.0,65.0,0.0000


In [64]:
school_tru=tru15[['Cds_Code','Cumulative_Enrollment','Truancy_Rate']]
school_tru['year']='2015'
school_tru.head()

/Users/libbyalthea/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Cds_Code,Cumulative_Enrollment,Truancy_Rate,year
0,1100170000000,5411.0,31.3065,2015
1,1316090000000,65.0,0.0000,2015
2,1316170000000,429.0,23.3100,2015
3,1611190000000,11461.0,22.8339,2015
4,1611270000000,4040.0,12.7227,2015


Now, combine into one final data frame

school_teacher

school_grads

apr_15

school_tru

In [81]:
#with apr as base
q3  = """
SELECT *
FROM
    apr15 apr
LEFT JOIN school_teacher st
    ON apr.cds=st.CDS
    AND apr.year=st.year
LEFT JOIN school_grads sg
    ON apr.cds=sg.cds
    AND apr.year=sg.year
LEFT JOIN school_tru tru
    ON apr.cds=tru.CDs_Code
    AND apr.year=tru.year
LEFT JOIN frpm15 f
    ON apr.cds=f.cds
"""


In [82]:
df= pysqldf(q3)
df.head()

KeyboardInterrupt: 

In [67]:
df[df.Cumulative_Enrollment>10000]

,sname,cds,type,charter,met_all,crit1,crit2,e_enr,e_tst,e_prof,...,Cumulative_Enrollment,Truancy_Rate,year,County_Code,District_Code,School_Code,perc_eligible_free,perc_eligible_FRPM,year,cds
0,State Total,0,None,None,Yes,26,26,3298149.0,3205026.0,1354011,...,6418044.0,31.4308,2015,None,None,None,NaN,NaN,None,None
14,None,1611190000000,1,None,No,20,22,4657.0,4495.0,2768,...,11461.0,22.8339,2015,None,None,None,NaN,NaN,None,None
43,None,1611430000000,1,None,No,18,20,4999.0,4683.0,2656,...,10853.0,12.9549,2015,None,None,None,NaN,NaN,None,None
79,None,1611760000000,1,None,Yes,22,22,18166.0,17795.0,12341,...,35535.0,27.5531,2015,None,None,None,NaN,NaN,None,None
121,None,1611920000000,1,None,No,22,24,11404.0,11095.0,3451,...,23915.0,49.9602,2015,None,None,None,NaN,NaN,None,None
158,None,1612000000000,1,None,Yes,22,22,6620.0,6470.0,3676,...,14051.0,13.5435,2015,None,None,None,NaN,NaN,None,None
193,None,1612420000000,1,None,No,22,24,6481.0,6269.0,2699,...,12940.0,38.5085,2015,None,None,None,NaN,NaN,None,None
207,None,1612590000000,1,None,No,22,24,19147.0,18161.0,5048,...,50715.0,30.2060,2015,None,None,None,NaN,NaN,None,None
348,None,1613090000000,1,None,No,20,22,5782.0,5488.0,1427,...,12695.0,38.2906,2015,None,None,None,NaN,NaN,None,None
379,None,1751010000000,1,None,Yes,22,22,8132.0,7992.0,6221,...,15255.0,20.8521,2015,01,75101,0000000,0.083333,0.125000,2015,01751010000000


In [68]:
df.count()

sname                     9964
cds                      11046
type                     11045
charter                   1173
met_all                  11046
crit1                    11046
crit2                    11046
e_enr                    10907
e_tst                    10907
e_prof                   10907
m_enr                    10907
m_tst                    10907
m_prof                   10907
ee_aa                    10907
et_aa                    10907
enp_aa                   11046
me_aa                    10907
mt_aa                    10907
mnp_aa                   11046
ee_wh                    10907
et_wh                    10907
enp_wh                   11046
me_wh                    10907
mt_wh                    10907
mnp_wh                   11046
ee_as                    10907
et_as                    10907
enp_as                   11046
me_as                    10907
mt_as                    10907
                         ...  
SchAttend                 3013
MetAtten

In [83]:
#with school as base
q4  = """
SELECT *
FROM school s
JOIN apr15 apr
    ON s.CDScode=apr.cds
LEFT JOIN school_teacher st
    ON s.CDScode=st.CDS
LEFT JOIN school_grads sg
    ON s.CDScode=sg.cds
LEFT JOIN school_tru tru
    ON s.CDScode=tru.CDs_Code
LEFT JOIN frpm15 f
    ON s.CDScode=f.cds
LEFT JOIN income i
    ON s.five_zip=i.Zip
LEFT JOIN school_enr e
    ON s.CDScode=e.cds


"""


In [84]:
df_final= pysqldf(q4)
df_final.count()

ClosedDate                  47
CDSCode                  10903
StatusType               10903
County                   10903
District                 10903
School                    9826
Zip                      10903
Latitude                 10903
Longitude                10903
FundingType               1154
Virtual                   9826
ClosedYear                  47
five_zip                 10903
sname                     9826
cds                      10903
type                     10903
charter                   1120
met_all                  10903
crit1                    10903
crit2                    10903
e_enr                    10765
e_tst                    10765
e_prof                   10765
m_enr                    10765
m_tst                    10765
m_prof                   10765
ee_aa                    10765
et_aa                    10765
enp_aa                   10903
me_aa                    10765
                         ...  
total_grads                115
uc_grads

In [85]:
df_final.to_csv('df.csv', sep='\t', encoding='utf-8')

In [72]:
df=pd.read_csv('df.csv', sep='\t')


/Users/libbyalthea/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
#df.drop(['cds.1','year.1','year.2','Cds_Code','year.3','total_grads','uc_grads','male_grads','male_uc_grads','minority_grads','minority_uc_grads','Unnamed: 0'],axis=1,inplace=True)

In [74]:
df_final.shape

(11907, 96)

In [75]:
df.count()

Unnamed: 0               11907
ClosedDate                 114
CDSCode                  11907
StatusType               11907
County                   11907
District                 11907
School                   10732
Zip                      11907
Latitude                 11907
Longitude                11907
FundingType               1303
Virtual                  10657
ClosedYear                 114
five_zip                 11907
sname                     9826
cds                      10903
type                     10903
charter                   1120
met_all                  10903
crit1                    10903
crit2                    10903
e_enr                    10765
e_tst                    10765
e_prof                   10765
m_enr                    10765
m_tst                    10765
m_prof                   10765
ee_aa                    10765
et_aa                    10765
enp_aa                   10903
                         ...  
CDS                       7883
SchoolCo